## Python, четвертое практическое занятие
1. Исключения и их обработка
2. Итераторы и генераторы
3. Comprehensive Python
4. Элементы функционального программирования: лямбда-выражения и функции высшего порядка

## 1. Исключения и их обработка
При выполнении программ могут возникать исключительные ситуации, которые надо уметь обрабатывать. Например, открытие несуществующего файла на чтение:

In [1]:
f = open('does_not_exists', 'r')

IOError: [Errno 2] No such file or directory: 'does_not_exists'

Задача разработчика состоит в корректной идентификации таких случаев и их обработке. Например, если вы разрабатываете функцию, которая должна принимать два натуральных числа, можно инициировать исключение для неправильных аргументов:

In [2]:
def positive_product(x, y):
    if not isinstance(x, int) or not isinstance(y, int):
        raise TypeError('Arguments should be integers.')
    if x <= 0 or y <= 0:
        raise ValueError('Arguments should be positive.')
    return x * y

Тогда при вызове функции с некорректными параметрами будут происходить ошибки и выполнение программы завершится:

In [3]:
# первый аргумент не является числом
result = positive_product('string', 5)

TypeError: Arguments should be integers.

In [4]:
# первый аргумент не является натуральным числом
result = positive_product(-3, 1)

ValueError: Arguments should be positive.

Задача пользователя вашей функции состоит в том, чтобы отслеживать и корректно обрабатывать исключения с помощью конструкций **try-except**:

In [5]:
x = 5
y = 3
try:
    result = positive_product(x, y)
except TypeError as e:
    print(e.message)
    print('Wrong types: {} and {}.'.format(type(x), type(y)))
except ValueError as e:
    print(e.message)
    print('Wrong values: {} and {}'.format(x, y))
except StandardError as e:
    print(e.message)
else:
    print('Everything is fine.')
finally:
    print('This block is always executed.')

Everything is fine.
This block is always executed.


При обработке исключений обязательно нужно написать один **try** и хотя бы один **except**. Необязательный блок **else** выполняется, если в блоке **try** не возникло исключений, необязательный блок **finally** выполняется в любом случае. Довольно часто пишут несколько операторов **except** -- по одному на "предсказуемые" ошибки и последний для ошибок, появление которых предусмотреть не удалось.

Можно создавать собственные классы исключений:

In [6]:
class AlreadyInListException(Exception):
    def __init__(self, item, positions):
        self.message = 'Item {} already in list {}.'.format(item, positions)
        
        
def add_position(new_pos, positions):
    if new_pos in positions:
        raise AlreadyInListException(new_pos, positions)
    positions.append(new_pos)
    return positions


positions = [(1, 1), (3, 5)]
try:
    positions = add_position((3, 5), positions)
except AlreadyInListException as e:
    print(e.message)

Item (3, 5) already in list [(1, 1), (3, 5)].


# 2. Итераторы и генераторы
Мы можем перебирать элементы некоторых типов данных с помощью цикла **for** (списки, кортежи, строки, словари и т.д.). Такие объекты называются итерируемыми (**iterable**). Чтобы объект был итерируемым, в нём должен быть специальный метод \__**iter**\__. Проверить, является ли объект итерируемым, можно с помощью функции **iter**:

In [7]:
def is_iterable(x):
    try:
        it = iter(x)
    except TypeError:
        return False
    else:
        return True
    
    
print(is_iterable(42))
print(is_iterable([1, 2, 3]))
print(is_iterable('hello'))

False
True
True


Python позволяет создавать специальные классы генераторов. Аналогично классам итераторов, в генераторах должен содержаться специальный метод \__**iter**\__, кроме того, должен быть метод **next** (пример генератора целых положительных чисел до заданного n):

In [8]:
class GenRange(object):
    def __init__(self, n):
        self._current = 0
        self._n = n
        
    def __iter__(self):
        return self
    
    def next(self):
        if self._current == self._n:
            raise StopIteration()
        self._current = self._current + 1
        return self._current
    
    
for val in GenRange(10):
    print(val)

1
2
3
4
5
6
7
8
9
10


Недостатком генератора является то, что каждый элемент можно использовать только один раз; безусловным преимуществом является экономия памяти, т.к. в любой момент времени в генераторе находится только один элемент (и, возможно, вспомогательные переменные, которые не занимают много места). Без генераторов не обойтись в работе с большими данными (такими, которые не могут быть размещены в оперативной памяти компьютера).

Можно создавать бесконечные генераторы (генератор натуральных чисел):

In [9]:
class NaturalNumbers(object):
    def __init__(self):
        self._current = 0
        
    def __iter__(self):
        return self
    
    def next(self):
        self._current = self._current + 1
        return self._current
    
    
natural_numbers = NaturalNumbers()
print(next(natural_numbers))
print(next(natural_numbers))
print(next(natural_numbers))

1
2
3


Такой же генератор, только созданный через функцию, возвращающую результат оператором **yield**:

In [10]:
def function_natural_numbers():
    i = 1
    while True:
        yield i
        i = i + 1
        
        
natural_numbers2 = function_natural_numbers()
print(next(natural_numbers2))
print(next(natural_numbers2))
print(next(natural_numbers2))

1
2
3


## 3. Comprehensive Python

Для быстрой генерации один итерируемых структур из других в Python есть специальные методы, называемые list comprehensions. Рассмотрим некоторые примеры.

Генерация списка, состоящего из элементов другого списка, возведенных в квадрат:

In [11]:
some_list = range(10)
new_list = [el * el for el in some_list]
print(some_list)
print(new_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


Генерация словаря из другого словаря:

In [12]:
some_dict = {'key1': 1, 'key2': 2, 'key3': 3}
other_dict = {k: some_dict[k] * 2 for k in some_dict}
print(some_dict)
print(other_dict)

{'key3': 3, 'key2': 2, 'key1': 1}
{'key3': 6, 'key2': 4, 'key1': 2}


Генерация спиской ключей или значений из словаря:

In [13]:
dict_keys = [k for k in some_dict]
dict_values = [some_dict[k] for k in some_dict]
print(dict_keys)
print(dict_values)

['key3', 'key2', 'key1']
[3, 2, 1]


К слову, для последней операции (и не только) в словаре существуют специальные функции:
- .keys() - возвращает список ключей словаря
- .values() - возвращает список значений словаря
- .iterkeys() - возвращает итератор ключей словаря
- .itervalues() - возвращает итератор значений словаря
Также интересны функции, возвращающие пары "ключ-значение" для словаря:
- .items() - в виде списка
- .iteritems() - в виде итератора.

In [14]:
print('Список ключей: {}'.format(some_dict.keys()))
print('Список значений: {}'.format(some_dict.values()))
print('Итератор ключей: {}'.format(some_dict.iterkeys()))
print('Итератор значений: {}'.format(some_dict.itervalues()))
print('Список пар ключ/значение: {}'.format(some_dict.items()))
print('Итератор пар ключ/значение: {}'.format(some_dict.iteritems()))

Список ключей: ['key3', 'key2', 'key1']
Список значений: [3, 2, 1]
Итератор ключей: <dictionary-keyiterator object at 0x106027fc8>
Итератор значений: <dictionary-valueiterator object at 0x106027fc8>
Список пар ключ/значение: [('key3', 3), ('key2', 2), ('key1', 1)]
Итератор пар ключ/значение: <dictionary-itemiterator object at 0x1052bb1b0>


Пример генерации с условием (список генерируется только из чётных элементов):

In [15]:
some_list2 = [k * k for k in some_list if k % 2 == 0]
print(some_list)
print(some_list2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 4, 16, 36, 64]


## 4. Элементы функционального программирования
В Python существуют некоторые элементы функционального программирования, например, лямбда-выражения, которые позволяют создавать небольшие функции (в том числе анонимные) и функции высшего порядка, такие, как **map**, **zip**, **filter**, **reduce**.

Пример функции "традиционного" вида и того, как её можно переписать с помощью лямбда-выражения:

In [16]:
def square(x):
    return x * x


lam_square = lambda x: x * x

Пример использования лямбда-функции без определения (анонимная функция, определена только в одном месте):

In [17]:
new_list = [(lambda x: x * x)(el) for el in some_list]
print(some_list)
print(new_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


Выше определяется анонимная лямбда-функция, которая применяется ко всем элементам списка some_list.

Лямбда-выражения особенно удобно использовать в фукнциях высшего порядка.

Функция **map** принимает в качестве первого аргумента функцию от N переменных, остальные аргументы -- N итерируемых объектов одинаковой длины; функция, заданная первым аргументом, будет применяться поэлементно к остальным аргументам:

In [18]:
print(map(lambda x: x * x, [1, 2, 3]))

[1, 4, 9]


In [19]:
print(map(lambda x, y: x + y, [1, 2, 3], [4, 5, 6]))

[5, 7, 9]


In [20]:
print(map(lambda x, y: x * y, [1, 2], [4, 5, 6]))  # списки разной длины

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

Функция **filter** принимает функцию одного аргумента, возвращающую True или False, и итерируемый объект, к элементам которого применяется заданная функция. Результатом выполнения функции **filter** является список, составленный из элементов исходного итерируемого объекта, для которого заданная функция вернет True.

In [21]:
print(filter(lambda x: x % 2 == 0, range(10)))
print(filter(lambda x: x > 1 and x < 5, range(5, 15)))

[0, 2, 4, 6, 8]
[]


Функция **zip** принимает несколько итерируемых объектов (возможно, разной длины) и возвращает список кортежей; i-й кортеж содержит i-е элементы принятых объектов.

In [22]:
print(zip([1, 2, 3], [4, 5, 6]))
print(zip([1, 2], [3, 4, 5], 'hello', {'key': 'value'}))

[(1, 4), (2, 5), (3, 6)]
[(1, 3, 'h', 'key')]


Функция **reduce** служит для комбинации последовательности вычислений. В качестве аргументов она принимает функцию от двух аргументов (аккумулирующая переменная, текущая переменная), реализующую последовательность вычислений, итерируемый объект и начальное значение.

Пример вычисления произведения элементов в списке:

In [23]:
print(reduce(lambda res, x: res * x, [5, 6, 7], 1))

210


Пример объединения списка списков в один список:

In [24]:
print(reduce(lambda res, x: res + x, [[1, 2], [3, 4, 5]], []))

[1, 2, 3, 4, 5]


Стоит заметить, что в стандартной библиотеке Python есть модуль **itertools**, который содержит функции высшего порядка, возвращающие объекты-генераторы вместо списков:

In [25]:
from itertools import izip, imap, ifilter

In [26]:
greater_than_5 = ifilter(lambda x: x > 5, range(10))
print(greater_than_5)
for x in greater_than_5:
    print(x)

6
7
8
9


Интересной особенностью этих функций является то, что они могут быть применены к бесконечным объектам.

In [27]:
# квадраты натуральных чисел
nat_squares = imap(lambda x: x * x, function_natural_numbers())
# первые 5
for i in range(5):
    print(next(nat_squares))

1
4
9
16
25


In [28]:
# пары: натуральное число/его квадрат
natural_nums = function_natural_numbers()
nat_squares = imap(lambda x: x * x, function_natural_numbers())
pairs = izip(natural_nums, nat_squares)
# первые 5
for i in range(5):
    print(next(pairs))

(1, 1)
(2, 4)
(3, 9)
(4, 16)
(5, 25)


In [29]:
# только чётные натуральные числа
nat_evens = ifilter(lambda x: x % 2 == 0, function_natural_numbers())
# первые 5
for i in range(5):
    print(next(nat_evens))
print('-----')
# только нечётные натуральные числа
nat_odds = ifilter(lambda x: x % 2 != 0, function_natural_numbers())
# первые 5
for i in range(5):
    print(next(nat_odds))

2
4
6
8
10
-----
1
3
5
7
9


## Дополнительно: разбор задач со встречи

Задача: написать генератор, который способен перебрать все целые числа.

In [30]:
class AllInts(object):
    def __init__(self):
        self._current = 0
        self._next = 0
        self._negative = True
        
    def __iter__(self):
        return self
    
    def next(self):
        self._current = self._next
        if self._negative:
            self._next = -(self._current + 1)
        else:
            self._next = -self._current
        self._negative = not self._negative
        return self._current


ints = AllInts()
for i in range(11):
    print(next(ints))

0
-1
1
-2
2
-3
3
-4
4
-5
5


Задача: написать генератор списков длины N, состоящих из 0 и 1.

In [31]:
class ZerosAndOnes(object):
    def __init__(self, n):
        self._current = 0
        self._n = n
        
    def __iter__(self):
        return self
    
    def next(self):
        bins = bin(self._current)    # получаем двоичное представление
        bins = bins[2:]              # обрезаем символы '0b' в начале строки
        bins = map(int, list(bins))  # преобразовываем в список, затем в целый тип
        result = [0 for i in range(self._n - len(bins))]  # нули впереди списка
        result = result + bins
        self._current = self._current + 1
        if len(result) > self._n:
            raise StopIteration()
        else:
            return result
    

for x in ZerosAndOnes(3):
    print(x)

[0, 0, 0]
[0, 0, 1]
[0, 1, 0]
[0, 1, 1]
[1, 0, 0]
[1, 0, 1]
[1, 1, 0]
[1, 1, 1]
